# Pentagram Music Transformer Maker (ver. 1.0)

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

WARNING: This complete implementation is a functioning model of the Artificial Intelligence. Please excercise great humility, care, and respect. https://www.nscai.gov/

***

#### Project Los Angeles

#### Tegridy Code 2023

***

# GPU check

In [ ]:
!nvidia-smi

# Setup environment

In [ ]:
!git clone https://github.com/asigalov61/tegridy-tools

In [ ]:
!pip install --upgrade pip
!pip install torch==2.0.1
!pip install torch-summary
#!pip install sklearn
!pip install tqdm
!pip install matplotlib

In [ ]:
# Load modules and make data dir

print('Loading modules...')

import os
import pickle
import random
import secrets
import tqdm
import time
import math
from contextlib import nullcontext

import matplotlib.pyplot as plt

from torchsummary import summary
from sklearn import metrics

import torch

%cd /content/tegridy-tools/tegridy-tools/nanoGPT

from nanoGPT import GPTConfig, GPT

%cd /content/

%cd /content/tegridy-tools/tegridy-tools/

import TMIDIX

%cd /content/

if not os.path.exists('/content/INTS'):
    os.makedirs('/content/INTS')

print('Done')

# Load training data

In [ ]:
CHUNKS_LENGTH = 4096+4 # must be larger than model seq_len and must be divisible by 5
MIN_NUMBER_OF_CHUNK_EVENTS = 515 # min number of tokens per chunk
PAD_IDX = 1670 # model pad index

dataset_addr = "/content/INTS"

filez = list()
for (dirpath, dirnames, filenames) in os.walk(dataset_addr):
    filez += [os.path.join(dirpath, file) for file in filenames]
print('=' * 70)

filez.sort()

print('Loading training data... Please wait...')
print('=' * 70)

train_data = []

chunks_counter = 0
discarted_chunks_counter = 0

for f in tqdm.tqdm(filez):
  train_d = pickle.load(open(f, 'rb'))
  random.shuffle(train_d)
  for t in train_d:

    for i in range(0, len(t), CHUNKS_LENGTH):

      if 0 <= max(t[i:i+CHUNKS_LENGTH]) < PAD_IDX: # final data integrity check

        if len(t[i:i+CHUNKS_LENGTH]) == CHUNKS_LENGTH:
          train_data.append(t[i:i+CHUNKS_LENGTH])

        else:
          if len(t[i:i+CHUNKS_LENGTH]) > MIN_NUMBER_OF_CHUNK_EVENTS:
            td = t[i:i+CHUNKS_LENGTH] + [PAD_IDX] * (CHUNKS_LENGTH-len(t[i:i+CHUNKS_LENGTH])) # padding with pad index
            train_data.append(td)
          else:
            discarted_chunks_counter += 1

        chunks_counter += 1

print('=' * 70)
print('Total number of imput chunks:', chunks_counter)
print('Total number of good chunks:', len(train_data))
print('Total number of discarted chunks:', discarted_chunks_counter, '/', round(100 * discarted_chunks_counter/chunks_counter, 3), '%')
print('All data is good:', len(max(train_data, key=len)) == len(min(train_data, key=len)))
print('=' * 70)
print('Final data randomization...')
random.shuffle(train_data)
print('Done!')
print('=' * 70)

In [ ]:
len(train_data)

In [ ]:
train_data[0][:10], train_data[0][-10:]

In [ ]:
len(train_data) // 128

# Setup model

# Train

In [ ]:
# ------------------------------------------------------------------------------

SEQ_LEN = 4096
BATCH_SIZE = 32
NUM_EPOCHS = 1
GRADIENT_ACCUMULATE_EVERY = 4

NUM_BATCHES = (len(train_data) // BATCH_SIZE // GRADIENT_ACCUMULATE_EVERY) * NUM_EPOCHS

SAVE_EVERY = 200
PRINT_STATS_EVERY = 5
#VALIDATE_EVERY  = 100
#GENERATE_EVERY  = 200
#GENERATE_LENGTH = 32

eval_iters = 4
eval_only = False # if True, script exits right after the first eval
always_save_checkpoint = True # if True, always save a checkpoint after each eval

init_from = 'scratch' # 'scratch' or 'resume' or 'gpt2*'

out_dir = '/content/OUT'

# model

n_layer = 32
n_head = 32
n_embd = 1024
dropout = 0.0 # for pretraining 0 is good, for finetuning try 0.1+
bias = False # do we use bias inside LayerNorm and Linear layers?

# adamw optimizer
learning_rate = 2e-4 # max learning rate

grad_clip = 1.0 # clip gradients at this value, or disable if == 0.0


# system
device = 'cuda' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1' etc., or try 'mps' on macbooks
#dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler
dtype = 'float16'
compile = False # use PyTorch 2.0 to compile the model to be faster

#-------------------------------------------------------------------------------
eval_interval = SAVE_EVERY
log_interval = PRINT_STATS_EVERY

# data

gradient_accumulation_steps = GRADIENT_ACCUMULATE_EVERY # used to simulate larger batch sizes
batch_size = BATCH_SIZE # if gradient_accumulation_steps > 1, this is the micro-batch size
block_size = SEQ_LEN

# -----------------------------------------------------------------------------
config_keys = [k for k,v in globals().items() if not k.startswith('_') and isinstance(v, (int, float, bool, str))]
#exec(open('configurator.py').read()) # overrides from command line or config file
config = {k: globals()[k] for k in config_keys} # will be useful for logging
# -----------------------------------------------------------------------------

tokens_per_iter = gradient_accumulation_steps * batch_size * block_size
print(f"tokens per iteration will be: {tokens_per_iter:,}")

os.makedirs(out_dir, exist_ok=True)

torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
# note: float16 data type will automatically use a GradScaler
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)


# poor man's data loader
def get_batch(train_data, index):

    ix = train_data[index*batch_size:(index*batch_size)+batch_size]
    x = torch.stack([torch.LongTensor(i[:block_size]) for i in ix])
    y = torch.stack([torch.LongTensor(i[1:block_size+1]) for i in ix])
    if device_type == 'cuda':
        # pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
        x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
    else:
      x, y = x.to(device), y.to(device)
    return x, y

# init these up here, can override if init_from='resume' (i.e. from a checkpoint)
iter_num = 0

# model init
model_args = dict(n_layer=n_layer, n_head=n_head, n_embd=n_embd, block_size=block_size,
                  bias=bias, vocab_size=None, dropout=dropout, ignore_idx=PAD_IDX) # start with model_args from command line


if init_from == 'scratch':
    # init a new model from scratch
    print("Initializing a new model from scratch")
    # determine the vocab size we'll use for from-scratch training

    model_args['vocab_size'] = PAD_IDX+1
    gptconf = GPTConfig(**model_args)
    model = GPT(gptconf)

model = torch.nn.DataParallel(model)

model.to(device)

# initialize a GradScaler. If enabled=False scaler is a no-op
scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))

# optimizer
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.95

optimizer = model.module.configure_optimizers(weight_decay, learning_rate, (beta1, beta2), device_type)

train_losses = []
train_accs = []

val_losses = []
val_accs = []

# helps estimate an arbitrarily accurate loss over either split using many batches
@torch.no_grad()
def estimate_loss():
    out = {}
    out1 = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        accs = torch.zeros(eval_iters)
        for k in range(eval_iters):
            bidx = random.randint(0, NUM_BATCHES)
            X, Y = get_batch(train_data, bidx)
            with ctx:
                logits, loss, acc = model(X, Y, compute_acc=True)
            losses[k] = loss.mean().item()
            accs[k] = acc.mean().item()
        out[split] = losses.mean()
        out1[split] = accs.mean()
    model.train()
    return out, out1

# training loop
X, Y = get_batch(train_data, 0) # fetch the very first batch
batch_num = 1
t0 = time.time()
local_iter_num = 0 # number of iterations in the lifetime of this process
running_mfu = -1.0
for i in tqdm.tqdm(range(NUM_BATCHES)):

    # evaluate the loss on train/val sets and write checkpoints
    if iter_num % eval_interval == 0:
        losses, accs = estimate_loss()
        print(f"step {iter_num}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        print(f"step {iter_num}: train acc {accs['train']:.4f}, val acc {accs['val']:.4f}")

        val_losses.append(losses['val'].item())
        val_accs.append(accs['val'].item())


        best_val_loss = losses['val']
        best_val_acc = accs['val']
        if iter_num > 0:
            checkpoint = {
                'model': model.state_dict(),
                'model_args': model_args,
                'iter_num': iter_num,
                'best_val_loss': best_val_loss,
                'best_val_acc': best_val_acc,
                'train/loss': losses['train'],
                'val/loss': losses['val'],
                'train/acc': accs['train'],
                'val/acc': accs['val'],
                'lr': learning_rate,
                'config': config,
            }
            print(f"saving checkpoint to {out_dir}")
            torch.save(checkpoint, os.path.join(out_dir, 'model_checkpoint_'+str(iter_num)+'_steps_'+str(best_val_loss.item())+'_loss_'+str(best_val_acc.item())+'_acc.pth'))

    # forward backward update, with optional gradient accumulation to simulate larger batch size
    # and using the GradScaler if data type is float16
    for micro_step in range(gradient_accumulation_steps):
        with ctx:
            logits, loss, acc = model(X, Y, compute_acc=True)

            loss = loss / gradient_accumulation_steps # scale the loss to account for gradient accumulation

        # immediately async prefetch next batch while model is doing the forward pass on the GPU
        X, Y = get_batch(train_data, batch_num)
        batch_num += 1
        # backward pass, with gradient scaling if training in fp16
        scaler.scale(loss).backward(torch.ones(loss.shape).cuda())
    # clip the gradient
    if grad_clip != 0.0:
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
    # step the optimizer and scaler if training in fp16
    scaler.step(optimizer)
    scaler.update()
    # flush the gradients as soon as we can, no need for this memory anymore
    optimizer.zero_grad(set_to_none=True)

    # timing and logging
    t1 = time.time()
    dt = t1 - t0
    t0 = t1
    if iter_num % log_interval == 0:
        # get loss as float. note: this is a CPU-GPU sync point
        # scale up to undo the division above, approximating the true total loss (exact would have been a sum)
        accf = acc.mean().item() #* gradient_accumulation_steps
        lossf = loss.mean().item() * gradient_accumulation_steps

        train_losses.append(lossf)
        train_accs.append(accf)
        print(f"iter {iter_num}: loss {lossf:.4f}, acc {accf:.4f}, time {dt*1000:.2f}ms")
    iter_num += 1
    local_iter_num += 1

# Final Save

In [ ]:
out_dir = '/content/'

best_val_loss = losses['val']
best_val_acc = accs['val']
if iter_num > 0:
    checkpoint = {
        'model': model.state_dict(),
        'model_args': model_args,
        'iter_num': iter_num,
        'best_val_loss': best_val_loss,
        'best_val_acc': best_val_acc,
        'train/loss': losses['train'],
        'val/loss': losses['val'],
        'train/acc': accs['train'],
        'val/acc': accs['val'],
        'lr': learning_rate,
        'config': config,
    }
    print(f"saving checkpoint to {out_dir}")
    torch.save(checkpoint, os.path.join(out_dir, 'Pentagram_Music_Transformer_Small_Trained_Model_'+str(iter_num)+'_steps_'+str(round(best_val_loss.item(), 4))+'_loss_'+str(round(best_val_acc.item(), 4))+'_acc.pth'))
print('Done!')

In [ ]:
data = [train_losses, train_accs, val_losses, val_accs]

TMIDIX.Tegridy_Any_Pickle_File_Writer(data, '/content/losses_accuracies')

In [ ]:
# Save training loss graph

plt.plot([i for i in range(len(train_losses))] ,train_losses, 'b')
plt.savefig('/content/training_loss_graph.png')
plt.close()
print('Done!')

# Save training acc graph

plt.plot([i for i in range(len(train_accs))] ,train_accs, 'b')
plt.savefig('/content/training_acc_graph.png')
plt.close()
print('Done!')

# Save validation loss graph

plt.plot([i for i in range(len(val_losses))] ,val_losses, 'b')
plt.savefig('/content/validation_loss_graph.png')
plt.close()
print('Done!')

# Save validation acc graph

plt.plot([i for i in range(len(val_accs))] ,val_accs, 'b')
plt.savefig('/content/validation_acc_graph.png')
plt.close()
print('Done!')

# Eval

In [ ]:
out_dir = '/content/OUT'
# model init
model_args = dict(n_layer=0, n_head=0, n_embd=0, block_size=0,
                  bias=False, vocab_size=0, dropout=0, ignore_idx=1563) # start with model_args from command line


print(f"Restoring checkpoint from {out_dir}")
# resume training from a checkpoint.
ckpt_path = os.path.join(out_dir, 'model_checkpoint.pth')
checkpoint = torch.load(ckpt_path, map_location='cuda')
checkpoint_model_args = checkpoint['model_args']
# force these config attributes to be equal otherwise we can't even resume training
# the rest of the attributes (e.g. dropout) can stay as desired from command line
for k in ['n_layer', 'n_head', 'n_embd', 'block_size', 'bias', 'vocab_size']:
    model_args[k] = checkpoint_model_args[k]
# create the model
gptconf = GPTConfig(**model_args)
model = GPT(gptconf)
state_dict = checkpoint['model']
# fix the keys of the state dictionary :(
# honestly no idea how checkpoints sometimes get this prefix, have to debug more
unwanted_prefix = '_orig_mod.'
for k,v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
model.load_state_dict(state_dict)
iter_num = checkpoint['iter_num']
best_val_loss = checkpoint['best_val_loss']
print('Done!')

In [ ]:
import statistics


f = '/content/Pentagram-Music-Transformer-MI-Seed-1.mid'

print('=' * 70)
print('Pentagram Music Transformer Seed MIDI Loader')
print('=' * 70)
print('Loading seed MIDI...')
print('=' * 70)
print('File:', f)
print('=' * 70)

#=======================================================
# START PROCESSING

# Convering MIDI to ms score with MIDI.py module
score = TMIDIX.midi2single_track_ms_score(open(f, 'rb').read(), recalculate_channels=False)

# INSTRUMENTS CONVERSION CYCLE
events_matrix = []
itrack = 1
patches = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

while itrack < len(score):
    for event in score[itrack]:
        if event[0] == 'note' or event[0] == 'patch_change':
            events_matrix.append(event)
    itrack += 1

events_matrix.sort(key=lambda x: x[1])

events_matrix1 = []

for event in events_matrix:
        if event[0] == 'patch_change':
              patches[event[2]] = event[3]

        if event[0] == 'note':
              event.extend([patches[event[3]]])

              events_matrix1.append(event)

if len(events_matrix1) > 0:
  if min([e[1] for e in events_matrix1]) >= 0 and min([e[2] for e in events_matrix1]) >= 0:

    #=======================================================
    # PRE-PROCESSING

    # checking number of instruments in a composition
    instruments_list_without_drums = list(set([y[3] for y in events_matrix1 if y[3] != 9]))
    instruments_list = list(set([y[3] for y in events_matrix1]))

    if len(events_matrix1) > 0 and len(instruments_list_without_drums) > 0:

      # recalculating timings
      for e in events_matrix1:
          e[1] = int(e[1] / 8) # Max 2 seconds for start-times
          e[2] = int(e[2] / 16) # Max 4 seconds for durations

      # Sorting by pitch, then by start-time
      events_matrix1.sort(key=lambda x: x[4], reverse=True)
      events_matrix1.sort(key=lambda x: x[1])

      #=======================================================
      # FINAL PRE-PROCESSING

      melody_chords = []

      pe = events_matrix1[0]

      for e in events_matrix1:

          # Cliping all values...
          time = max(0, min(255, e[1]-pe[1]))
          dur = max(0, min(255, e[2]))
          cha = max(0, min(15, e[3]))
          ptc = max(1, min(127, e[4]))

          # Calculating octo-velocity
          vel = max(1, min(127, e[5]))

          pat = max(0, min(127, e[6]))

          # Writing final note
          melody_chords.append([time, dur, cha, ptc, vel, pat])

          pe = e



      #=======================================================
      # FINAL PROCESSING
      #=======================================================

      melody_chords2 = []

      # Break between compositions / Intro seq

      if 9 in instruments_list:
        drums_present = 1539 # Yes
      else:
        drums_present = 1538 # No

      if melody_chords[0][2] != 9:
          pat = melody_chords[0][5]
      else:
          pat = 128

      melody_chords2.extend([1669, 1669, 1669, drums_present, 1540+pat])

      #=======================================================

      # TOTAL DICTIONARY SIZE 1669+1=1670

      #=======================================================
      # MAIN PROCESSING CYCLE
      #=======================================================

      note_counter = 1

      for m in melody_chords:

          if note_counter % 100 == 0:
              nct = 1025+min(511, (note_counter // 100)) # note counter token
              melody_chords2.extend([nct, nct, nct, nct, nct])

          if m[2] != 9:
              ptc = m[3]
              pat = m[5]
          else:
              ptc = m[3] + 128
              pat = 128

          melody_chords2.extend([pat, m[0]+129, m[1]+385, ptc+641, m[4]+897])

          note_counter += 1

#=======================================================

In [ ]:
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda'
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler
dtype = 'float16'
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)
model.cuda()
model.eval()

#x = (torch.tensor(random.choice(train_data)[:1000], dtype=torch.long, device=device_type)[None, ...])
#x = torch.tensor([[1669, 1669, 1669, 1538+1, 1540+40]], dtype=torch.long, device=device_type)
x = torch.tensor([[1669, 1669, 1669]], dtype=torch.long, device=device_type)

#x = torch.tensor([melody_chords2[:1000]], dtype=torch.long, device=device_type)


# run generation
#with torch.no_grad():
with ctx:
#        for k in range(1):
  y = model.module.generate(x, 1000, temperature=0.85, return_prime=False)
  #print(y[0].tolist())
  print('---------------')

In [ ]:
len(melody_chords2)

In [ ]:
#@title Test INTs

data = y[0].tolist()

print('Sample INTs', data[:15])

out = data[:200000]

if len(out) != 0:

    song = out
    song_f = []
    time = 0
    dur = 0
    vel = 90
    pitch = 0
    channel = 0

    patches = [0] * 16

    channels = [0] * 16
    channels[9] = 1


    for ss in song:

      if ss >= 0 and ss <= 128:

        if ss < 128:

            if ss not in patches:
              cha = channels.index(0)
              channels[cha] = 1

              patches[cha] = ss
              channel = patches.index(ss)
            else:
              channel = patches.index(ss)

        if ss == 128:
            channel = 9

      if ss > 128 and ss <= 256:

        time += (ss-129) * 8

      if ss > 384 and ss <= 640:

        dur = (ss-385) * 16

      if ss > 640 and ss <= 896:

        pitch = (ss-641) % 128

      if ss > 896 and ss <= 1024:

        vel = (ss-897)

        song_f.append(['note', time, dur, channel, pitch, vel ])

    detailed_stats = TMIDIX.Tegridy_SONG_to_MIDI_Converter(song_f,
                                                        output_signature = 'Pentagram Music Transformer',
                                                        output_file_name = '/content/Pentagram-Music-Transformer-Composition',
                                                        track_name='Project Los Angeles',
                                                        list_of_MIDI_patches=patches,
                                                        number_of_ticks_per_quarter=500)

In [ ]:
patches

In [ ]:
tok_emb = model.module.transformer.wte.weight.detach().cpu().tolist()

cos_sim = metrics.pairwise_distances(
  tok_emb, metric='cosine'
)
plt.figure(figsize=(7, 7))
plt.imshow(cos_sim, cmap="inferno", interpolation="nearest")
im_ratio = cos_sim.shape[0] / cos_sim.shape[1]
plt.colorbar(fraction=0.046 * im_ratio, pad=0.04)
plt.xlabel("Position")
plt.ylabel("Position")
plt.tight_layout()
plt.plot()
plt.savefig("/content/Pentagram-Music-Transformer-Tiny-Tokens-Embeddings-Plot.png", bbox_inches="tight")

# Congrats! You did it! :)